In [ ]:
!pip install PyPDF2 langchain nltk openai requests faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import re
import nltk
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import openai
import requests
import numpy as np
import faiss
from google.colab import drive
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def extract_text_from_pdfs(pdf_paths):
    text = ""
    for pdf_path in pdf_paths:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)  # Use PdfReader instead of PdfFileReader
            for page_num in range(len(reader.pages)):  # Iterate through pages using reader.pages
                page = reader.pages[page_num]
                text += page.extract_text()
    return text

# Example PDF paths
pdf_dir = '/content/drive/MyDrive/ChatBot/Dataset'  # Replace with your Google Drive folder path
pdf_paths = [os.path.join(pdf_dir, pdf) for pdf in os.listdir(pdf_dir) if pdf.endswith('.pdf')]

text = extract_text_from_pdfs(pdf_paths)
print(text[:1000])  # Print the first 1000 characters to verify

   
The Family Therapy  
Treatment Planner,  
with DSM-5 Updates, 
Second Edition Practice Planners® Series 
Treatment Planners 
The Complete Adult Psychotherapy Treatment Planner, Fifth Edition 
The Child Psychotherapy Treatment Planner, Fifth Edition 
The Adolescent Psychotherapy Treatment Planner, Fifth Edition 
The Addiction Treatment Planner, Fifth Edition 
The Continuum of Care Treatment Planner 
The Couples Psychotherapy Treatme nt Planner, with DSM-5 Updates, Second Edition 
The Employee Assistance Treatment Planner 
The Pastoral Counseling Treatment Planner 
The Older Adult Psychotherapy Trea tment Planner, with DSM-5 Updates, Second Edition
The Behavioral Medicine  Treatment Planner 
The Group Therapy Treatment Planner 
The Gay and Lesbian Psychotherapy Treatment Planner 
The Family Therapy Treatment Planner, with DSM-5 Updates, Second Edition
The Severe and Persistent Mental Illness Treatment Planner, with DSM-5 Updates, Second Edition
The Mental Retardation and Developmenta

In [ ]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    # text = re.sub(r'\d+', '', text)   # Remove numbers
    text = text.lower()  # Convert text to lowercase
    return text

In [ ]:
cleaned_text = clean_text(text)

In [ ]:
def split_into_chunks(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = split_into_chunks(cleaned_text)


In [ ]:
print(chunks[0])

 the family therapy treatment planner, with dsm-5 updates, second edition practice planners® series treatment planners the complete adult psychotherapy treatment planner, fifth edition the child psychotherapy treatment planner, fifth edition the adolescent psychotherapy treatment planner, fifth edition the addiction treatment planner, fifth edition the continuum of care treatment planner the couples psychotherapy treatme nt planner, with dsm-5 updates, second edition the employee assistance treatment planner the pastoral counseling treatment planner the older adult psychotherapy trea tment planner, with dsm-5 updates, second edition the behavioral medicine treatment planner the group therapy treatment planner the gay and lesbian psychotherapy treatment planner the family therapy treatment planner, with dsm-5 updates, second edition the severe and persistent mental illness treatment planner, with dsm-5 updates, second edition the mental retardation and developmental disability treatment

In [ ]:
import json

In [ ]:
# Save text chunks
chunks_path = '/content/drive/MyDrive/ChatBot/Output_Embeddings/chunks.json'
with open(chunks_path, 'w') as f:
    json.dump(chunks, f)
print(f"Chunks saved to {chunks_path}")

Chunks saved to /content/drive/MyDrive/ChatBot/Output_Embeddings/chunks.json


In [ ]:
# Set OpenAI API key
api_key = '..........'

In [ ]:
# # Reduce the number of chunks to avoid exceeding the quota
# chunks = chunks[:3]  # Adjust the number as needed for testing

In [ ]:
# Function to get embeddings using requests
def get_embeddings(text_chunks):
    embeddings = []
    url = "https://api.openai.com/v1/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    for i, chunk in enumerate(text_chunks):
        data = {
            "input": chunk,
            "model": "text-embedding-3-large"  # Use the model name that is available for embeddings
        }
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            embeddings.append(response.json()['data'][0]['embedding'])
            print(f"Chunk {i} processed successfully.")
        else:
            print(f"Error {response.status_code}: {response.text}")
    return embeddings

# Generate embeddings for text chunks
# Assuming chunks is your list of text chunks
# Generate embeddings for text chunks
embeddings = get_embeddings(chunks)

# Convert embeddings to numpy array and save to Google Drive
embeddings_array = np.array(embeddings).astype('float32')
embeddings_path = '/content/drive/MyDrive/ChatBot/Output_Embeddings/embeddings.npy'
np.save(embeddings_path, embeddings_array)
print(f"Embeddings saved to {embeddings_path}")


Streaming output truncated to the last 5000 lines.
Chunk 3263 processed successfully.
Chunk 3264 processed successfully.
Chunk 3265 processed successfully.
Chunk 3266 processed successfully.
Chunk 3267 processed successfully.
Chunk 3268 processed successfully.
Chunk 3269 processed successfully.
Chunk 3270 processed successfully.
Chunk 3271 processed successfully.
Chunk 3272 processed successfully.
Chunk 3273 processed successfully.
Chunk 3274 processed successfully.
Chunk 3275 processed successfully.
Chunk 3276 processed successfully.
Chunk 3277 processed successfully.
Chunk 3278 processed successfully.
Chunk 3279 processed successfully.
Chunk 3280 processed successfully.
Chunk 3281 processed successfully.
Chunk 3282 processed successfully.
Chunk 3283 processed successfully.
Chunk 3284 processed successfully.
Chunk 3285 processed successfully.
Chunk 3286 processed successfully.
Chunk 3287 processed successfully.
Chunk 3288 processed successfully.
Chunk 3289 processed successfully.
Chun

In [ ]:
# Load embeddings from Google Drive (in a new session)
loaded_embeddings = np.load(embeddings_path)
print(f"Embeddings loaded from {embeddings_path}")

# Create FAISS index
dimension = loaded_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(loaded_embeddings)

# Save the FAISS index to Google Drive
index_path = '/content/drive/MyDrive/ChatBot/faiss_index.bin'
faiss.write_index(index, index_path)
print(f"FAISS index saved to {index_path}")

Embeddings loaded from /content/drive/MyDrive/ChatBot/Output_Embeddings/embeddings.npy
FAISS index saved to /content/drive/MyDrive/ChatBot/faiss_index.bin


In [ ]:
loaded_embeddings[0]

array([-0.00651992, -0.00293231, -0.0239297 , ...,  0.01912903,
        0.00073952, -0.02388552], dtype=float32)

In [ ]:
dimension

3072

In [ ]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x795d0aeb2910> >